# DP0.2 Notebook 16 Galaxy Photometry

<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=250 style="padding: 10px" alt="Rubin Observatory logo, a graphical representation of turning stars into data.">
<br>
Contact author(s): <i>Christina Williams</i> <br>
Last verified to run: <i>2024-01-10</i> <br>
LSST Science Pipelines version: Weekly <i>2024_24</i> <br>
Container Size: <i>medium</i> <br>
Targeted learning level: <i>beginner</i> <br>

In [ ]:
#%load_ext pycodestyle_magic
#%flake8_on
#import logging
#logging.getLogger("flake8").setLevel(logging.FATAL)

**Description:** Explore the available measurements of galaxy photometry produced by the LSST pipelines and their applications.

**Skills:** Basic understanding of photometric measurement methods typically used to characterize galaxies.

**LSST Data Products:** objectTable

**Packages:** No new LSST packages are introduced here. The notebook will make use of the TAP service from the `lsst.rsp` package.

**Credit:**
"This notebook benefitted from earlier exploration of simulated data and notebook development by Melissa Graham and Dan Taranu. _E.g., "Originally developed by" or "Based on notebooks developed by" and then people's names, including journal article or software release citations if appropriate._
Please consider acknowledging them if this notebook is used for the preparation of journal articles, software releases, or other notebooks.

**Get Support:**
Find DP0-related documentation and resources at <a href="https://dp0.lsst.io">dp0.lsst.io</a>.
Questions are welcome as new topics in the 
<a href="https://community.lsst.org/c/support/dp0">Support - Data Preview 0 Category</a> 
of the Rubin Community Forum. 
Rubin staff will respond to all questions posted there.

## 1. Introduction

This notebook will teach the user about the automated photometry measurements that are measured on the deepCoadd images and appear in the Object Catalog as part of the LSST pipelines data products. The focus will be on galaxies (and thus is distinct from the demonstrations in Tutorial Notebook 12a/b about the PSF photometry (CHECK/LINK). 


### 1.1 Types of photometry measurements that exist in the Object Table

Schema for the object catalog for DP0.2: https://dm.lsst.org/sdm_schemas/browser/dp02.html#Object

Numerous photometry measurements are produced by the LSST Pipelines. Two types of photometry are there: The first are total fluxes are those that integrate the total light coming from objects, corrected for loss due to the blurring effects of PSF or seeing, including "Composite Model" (`cModel`) fluxes and "Kron Fluxes". The second category characterize the relative brightness of objects for specific regions of objects (i.e. are not corrected to be total fluxes) but can be useful for measuring accurate light profiles or accurate colors.

### 1.1.1 Total fluxes

#### Kron fluxes

A decent summary of Kron fluxes <a href="https://ned.ipac.caltech.edu/level5/March05/Graham/Graham2_6.html">in the NED documentation</a>.

```
<f>_kronFlux      : Flux from Kron Flux algorithm. Measured on <f> g-band.
<f>_kronFluxErr   : Uncertainty of <f>_kronFlux.
<f>_kronFlux_flag : Failure flag for <f>_kronFlux.
```

The Kron radius, `<f>_kronRad`, is also available.


#### Composite Model (CModel) fluxes

Similar in nature to those measured for SDSS: 
https://www.sdss3.org/dr8/algorithms/magnitudes.php#cmodel

In short, it is the linear combination of the best fit exponential and de Vaucouleurs profiles.

```
<f>_cModelFlux    :	Flux from the final cmodel fit. Forced on <f>-band.
<f>_cModelFluxErr : Uncertainty of <f>_cModelFlux
<f>_cModel_flag   : Failure flag for <f>_cModelFlux
```

Fluxes fit to the individual model components.

```
<f>_bdFluxB    : Flux from the de Vaucouleurs fit. Measured on <f>-band.
<f>_bdFluxD    : Flux from the exponential fit. Measured on <f>-band.
<f>_bdFluxBErr : Uncertainty of <f>_bdFluxB
<f>_bdFluxDErr : Uncertainty of <f>_bdFluxD
```

The fit sizes are also available (half-light radii, ellipse axes).

### 1.1.2 Regional fluxes

#### Aperture fluxes
This contains the enclosed flux inside a given aperture (these are raw fluxes, meaning, they are not corrected to total fluxes using an aperture correction that accounts for the flux falling outside the aperture using the PSF). Fixed aperture diameter size in pixels.

```
<f>_ap<pix>Flux     : Flux within <pix>-pixel aperture. Forced on <f>-band.
<f>_ap<pix>FluxErr  : Uncertainty of <f>_ap<pix>Flux.
<f>_ap<pix>FluxFlag : Failure flag for <f>_ap<pix>Flux.
```

For DP0.2, the apertures are 3, 6, 9, 12, 17, 25, 35, 50, and 70 pixels.

In the column name, apertures are `03`, `06`, `09`, `12`, and so on.

#### GaaP fluxes

These are the Gaussian-aperture-and-PSF flux from <a href="https://ui.adsabs.harvard.edu/abs/2008A%26A...482.1053K/abstract">Kuijken et al. 2008</a>. The main goal of this method is to measure accurate colors while accounting for the different spatial resolution between filters. This is sometimes done by convolving all images to the largest PSF, but this process is computationally very time consuming for large images. It is not a measure of total flux in a filter. For photometric redshifts, since accurate colors are important, it is the GaaP fluxes that should be used. Several apertures are available (TBD: what is optimal aperture based on?)

**Optimal**

```
<f>_gaapOptimalFlux    : GaaP flux with optimal aperture after multiplying the seeing by 1.15. Forced on <f>-band.
<f>_gaapOptimalFluxErr : Uncertainty of <f>_gaapOptimalFlux.
```

**PSF**

```
<f>_gaapPsfFlux    : GaaP flux with PSF aperture after multiplying the seeing by 1.15. Forced on <f>-band.
<f>_gaapPsfFluxErr : Uncertainty of <f>_gaapPsfFlux.
```

**Aperture**

```
<f>_gaap<ap>Flux    : GaaP flux with <ap> aperture after multiplying the seeing by 1.15. Forced on <f>-band.
<f>_gaap<ap>FluxErr : Uncertainty of <f>_gaap<ap>Flux.
```

Where the apertures are 0.5, 0.7, 1.0, 1.5, 2.5, and 3.0.
In the column name `<ap>` appears as `0p5`, `0p7`, etc.


#### PSF fluxes

Fluxes derived using the model point-spread function (PSF) of the image. Since PSF fluxes are designed for unresolved sources (not the majority of galaxies) this notebook will not explore science applications for this type of photometry. Interested users can find more information in the PSF Tutorial notebooks <a href="https://dp0-2.lsst.io/_static/nb_html/DP02_12a_PSF_Data_Products.html">12a</a> and <a href="https://dp0-2.lsst.io/_static/nb_html/DP02_12b_PSF_Science_Demo.html">12b</a>.

```
<f>_psfFlux      : Flux derived from linear least-squares fit of PSF model. Forced on <f>-band.
<f>_psfFluxErr   : Uncertainty of <f>_psfFlux.
<f>_psfFlux_flag : Failure flag for <f>_psfFlux.

```


### 1.2 Import packages


`numpy` is a fundamental package for scientific computing with arrays in Python
(<a href="https://numpy.org">numpy.org</a>).

`matplotlib` is a comprehensive library for creating static, animated, and
interactive visualizations in Python 
(<a href="https://matplotlib.org/">matplotlib.org</a>; 
<a href="https://matplotlib.org/stable/gallery/index.html">matplotlib gallery</a>).

From the `lsst` package, modules for accessing the TAP service, the butler, and image display functions are imported (<a href="https://pipelines.lsst.io/">pipelines.lsst.io</a>).

In [ ]:
import matplotlib.pyplot as plt
#from astropy.wcs import WCS
#from astropy.visualization import make_lupton_rgb
from astropy import units as u
from astropy.coordinates import SkyCoord
import gc
import numpy as np
import sys

#import lsst.afw.display as afwDisplay
#from lsst.afw.image import MultibandExposure
#from lsst.daf.butler import Butler
from lsst.rsp import get_tap_service
#import lsst.geom as geom

from scipy import stats#.binned_statistic
from scipy.stats import sigmaclip


### 1.3 Define functions and parameters

_TBD if we can remove this_

_It is OK to rename the subsection to be more specific to the notebook, and/or to use sub-sub-sections like "1.2.1 Define global cosmological parameter values" or "1.2.2 Define a function to make an image cutout"._

_It is OK to remove this sub-section if it is not being used._

In [ ]:
plt.style.use('tableau-colorblind10')
#afwDisplay.setDefaultBackend('matplotlib')

## 2. Integrated photometry of galaxies using the LSST pipeline

This section will explore the total, integrated photometry measurements, and provide some guidance for which are optimal for certain science applications using galaxies. 


### 2.1 Initialize the TAP service

In [ ]:
service = get_tap_service("tap")

### 2.2 Object table photometry measurements

First see what is available in the object catalog by querying the tap_schema columns, and printing all the parameters available related to "Flux" measured in the i-band (as an example). The return shows also errors and flags associated with the photometric measurements outlined in section 1.1. 

In [ ]:
query = "SELECT column_name, datatype, description, unit " \
        "FROM tap_schema.columns " \
        "WHERE table_name = 'dp02_dc2_catalogs.Object'"

results = service.search(query).to_table()

In [ ]:
search_string = 'Flux'
band = 'i_'
for cname in results['column_name']:
    if (cname.find(search_string) > -1) and (cname.find(band) > -1):
        print(cname)

### 2.3 Compare (total) simulated photometry measured with the LSST pipelines to their input values

The following cells will query the truth tables to get the input fluxes in the DP0.2 simulated objects, matched to those measured from the simulated images using the LSST pipelines. This comparison provides a sense of how good the various photometry measurement methods are by comparing input flux to measured flux. This provides some insight into what regimes they may perform well vs poorly.

The below cells query the truth values for i-band mag, measured i-band mag values from the catalog, and a number of flags that are relevant to the quality of the galaxy photometric measurements. Specifically, request `fromBlend`, `detect_isIsolated`, and `i_blendedness` which will indicate how much blending may have occurred, how large the galaxies are, to help provide insight in cases of catastrophic outliers.

Further, to ensure good measurements, restrict to objects which are detected at S/N > 5 and where `detect_isPrimary` is true, to ensure unique objects are returned (for more information on the use of the `detect_isPrimary` flag, see <a href="https://dp0-2.lsst.io/_static/nb_html/DP02_10_Deblender_Data_Products.html">Tutorial Notebook 10</a> about deblender data products. Also, set the condition that `i_extendedness` = 1 in order to return a population of galaxies (i.e. remove stars as point sources). To expedite the search we will limit to a 0.1 degree region on the sky.

In [ ]:
query = "SELECT mt.id_truth_type, mt.match_objectId, ts.ra, ts.dec, ts.truth_type, ts.flux_i, ts.redshift, "\
        "obj.coord_ra, obj.coord_dec, obj.detect_fromBlend, obj.detect_isIsolated, obj.i_blendedness_flag, obj.i_blendedness, obj.i_extendedness, "\
        "obj.i_kronFlux, obj.i_kronFluxErr, obj.i_kronRad, obj.i_kronFlux_flag, obj.i_cModelFlux, obj.i_gaapOptimalFlux " + \
        "FROM dp02_dc2_catalogs.MatchesTruth AS mt "\
        "JOIN dp02_dc2_catalogs.TruthSummary AS ts ON mt.id_truth_type = ts.id_truth_type "\
        "JOIN dp02_dc2_catalogs.Object AS obj ON mt.match_objectId = obj.objectId "\
        "WHERE (obj.detect_isPrimary = 1) AND (obj.i_kronFlux/obj.i_kronFluxErr > 10) AND (obj.i_extendedness = 1) AND "\
        "CONTAINS(POINT('ICRS', obj.coord_ra, obj.coord_dec), CIRCLE('ICRS', 62.0, -37.0, 0.20)) = 1 "

In [ ]:
job = service.submit_job(query)
job.run()
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)

Print the results of the search query.

In [ ]:
results = job.fetch_result()
print(len(results))
tab = results.to_table()
tab

### 2.4 Exploring the input vs output photometry

This section will explore 3 photometric measurements that are relevant for galaxies: cModel, Kron and GaaP (as defined in Section 2.1). 

First, store the AB magnitudes to go with the fluxes extracted from the objectTable.

In [ ]:
truth_mag =  -2.50 * np.log10(tab['flux_i']) + 31.4

cmodel_mag = -2.50 * np.log10(tab['i_cModelFlux']) + 31.4
kron_mag = -2.50 * np.log10(tab['i_kronFlux']) + 31.4
gaap_mag = -2.50 * np.log10(tab['i_gaapOptimalFlux']) + 31.4


The next cell will compare the percent difference between the three different photometric measurements with the input (truth). Additionally a running median is calculated to help interpret the typical accuracy of the different methods. The median has the best overall percent accuracy for cModel. Additionally, the Gaussian and PSF photometry is highly inaccurate, as explained in Section 1.1 this is not meant to measure the total flux of galaxies, but rather to measure accurate colors using a sub-region of each galaxy. Deviation from truth for this measurement is thus expected.

In [ ]:
fig, ax = plt.subplots()

ax.plot(truth_mag, (tab['i_cModelFlux']-tab['flux_i'])/tab['flux_i'] * 100,'.',alpha=.1,label='cModel',color='r')
ax.plot(truth_mag, (tab['i_kronFlux']-tab['flux_i'])/tab['flux_i'] * 100,'.',alpha=.1,label='Kron',color='blue')
ax.plot(truth_mag, (tab['i_gaapOptimalFlux']-tab['flux_i'])/tab['flux_i'] * 100,'.',alpha=.1, label='gaapOptimal',color='green')

x = truth_mag
y = (tab['i_cModelFlux']-tab['flux_i'])/tab['flux_i'] *100
bins=np.arange(16,27,1)
bin_means, bin_edges, binnumber = stats.binned_statistic(x,
             y, statistic='median', bins=bins)
binctr = bin_edges[:-1] + (bin_edges[1:]-bin_edges[:-1])/2
plt.plot(binctr, bin_means, color='red', lw=2, label='cModel bin median',zorder=10)

y = (tab['i_kronFlux']-tab['flux_i'])/tab['flux_i'] *100
bin_means, bin_edges, binnumber = stats.binned_statistic(x,
             y, statistic='median', bins=bins)
binctr = bin_edges[:-1] + (bin_edges[1:]-bin_edges[:-1])/2
plt.plot(binctr, bin_means, color='blue', lw=2, label='kron bin median',zorder=10)

y = (tab['i_gaapOptimalFlux']-tab['flux_i'])/tab['flux_i'] * 100
bin_means, bin_edges, binnumber = stats.binned_statistic(x,
             y, statistic='median', bins=bins)
binctr = bin_edges[:-1] + (bin_edges[1:]-bin_edges[:-1])/2
plt.plot(binctr, bin_means, color='green', lw=2, label='GaaP Optimal bin median',zorder=10)#,color='green')

ax.axhline(0,linestyle='--')
ax.set_xlabel('True i-band Magnitude')
ax.set_ylabel('Flux Percent Accuracy (F$_{obs}$ - F$_{true}$) / F$_{true}$ x 100%')
ax.set_ylim([-100,100])
ax.set_xlim([14,27])
plt.legend()


Below, take a closer look at the differences in AB magnitude and how the overall distributions compare. While the systematic offset of GaaP is clear, the medians of Kron and cModel both compare well. These two plots indicate that there is excellent agreement between the cModel measurements and the input, at the level of < 0.03 magnitude at ABmag of < 25. Kron is also adequate overall, although at intermediate magnitudes (ABmag ~ 18-22) exhibits a slight underestimate of flux (overestimate in magnitude of order 0.1-0.2).  

Note that the input light profiles of simulated galaxies in DP0.2 are sersic profiles; thus it is perhaps not unexpected that modeling the light as sersic profiles by the LSST pipelines to produce cModel fluxes would perform the best.

In [ ]:
fig, (ax, ax2) = plt.subplots(ncols=2, nrows=1, width_ratios=[0.8, 0.2], figsize=(10, 6))
ylims = [-1.2,1.2]

ax.plot(truth_mag, (kron_mag-truth_mag),'.',alpha=.1,label='Kron',color='blue')
ax.plot(truth_mag, (gaap_mag-truth_mag),'.',alpha=.1, label='gaapOptimal',color='green')
ax.plot(truth_mag, (cmodel_mag-truth_mag),'.',alpha=.1,label='cModel',color='r')

ax2.hist((kron_mag-truth_mag),edgecolor='blue',orientation="horizontal", bins=np.linspace(ylims[0], ylims[1], 40),align='mid',histtype="step",stacked=True,fill=False)
ax2.hist((gaap_mag-truth_mag),edgecolor='green',orientation="horizontal", bins=np.linspace(ylims[0], ylims[1], 40),align='mid',histtype="step",stacked=True,fill=False)
ax2.hist((cmodel_mag-truth_mag),edgecolor='red',orientation="horizontal", bins=np.linspace(ylims[0], ylims[1], 40),align='mid',histtype="step",stacked=True,fill=False)
ax2.axhline(0,linestyle='--')

x = truth_mag
y = (cmodel_mag-truth_mag)

bin_means, bin_edges, binnumber = stats.binned_statistic(x,
             y, statistic='median', bins=bins)
binctr = bin_edges[:-1] + (bin_edges[1:]-bin_edges[:-1])/2
ax.plot(binctr, bin_means, color='red', lw=2, label='cModel bin median',zorder=10)
for i in range(len(bin_means)):
    print(binctr[i],bin_means[i],'cModel')

    
y = (kron_mag-truth_mag)
bin_means, bin_edges, binnumber = stats.binned_statistic(x,
             y, statistic='median', bins=bins)
binctr = bin_edges[:-1] + (bin_edges[1:]-bin_edges[:-1])/2
ax.plot(binctr, bin_means, color='blue', lw=2, label='kron bin median',zorder=10)
for i in range(len(bin_means)):
    print(binctr[i],bin_means[i],'kron')

y = (gaap_mag-truth_mag)
bin_means, bin_edges, binnumber = stats.binned_statistic(x,
             y, statistic='median', bins=bins)
binctr = bin_edges[:-1] + (bin_edges[1:]-bin_edges[:-1])/2
ax.plot(binctr, bin_means, color='green', lw=2, label='GaaP Optimal bin median',zorder=10)#,color='green')

ax.axhline(0,linestyle='--')
ax.set_xlabel('True i-band Magnitude')
ax.set_ylabel('Magnitude difference (M$_{obs}$ - M$_{true}$)')
ax.set_xlim([14,27])
ax.set_ylim(ylims)
ax.legend()


The next cells will explore and compare the Kron magnitudes to truth under various circumstances. 2 basic paramters are set: extendedness = 1 means the source is a galaxy; kronFlag = 0 identifies galaxies for which there were no problems flagged in the Kron flux measurement. 

Test whether the following blending parameters indicate that blending contributes to the inaccurate measurement of the photometry:

1) detect_fromBlend = 1 means some deblending happened prior to flux measurement. This is responsible for some under-estimated magnitude (over-estimated flux) perhaps because neighbors were not removed at the bright end?

2) The increased scatter at faint magnitudes is due to increased fraction of flux contaminated by neighbors i_blendedness: (1 - child_flux/parent_flux)
   
3) detect_isIsolated = 1 means no deblending and the galaxy has no blended neighbors. The upturn at bright mags is not there among isolated galaxies

4) Not sure i understand the tail to negative flux difference (i.e. underestimated flux) at bright mags

In [ ]:
# here add a second set of panels to try to understand the small bias at bright end of kron (is it kron radius-dependent? since its not deblending dependent?


fig,(ax, ax2) = plt.subplots(1, 2,figsize=(12,5))
#whblend = np.where( tab['detect_fromBlend'][whbright] == False)[0]

# Pick things where there was no flagged error in the Kron meausrement, and things that are extended (i.e. galaxies)

# Remove i_extendedness, you put it directly into the query instead

whgood = np.where((tab['i_kronFlux_flag'] == 0) & ( tab['detect_fromBlend'] == 1))[0]

cmap = tab['i_blendedness'][whgood] #only saw things blended with 1 neighbor is this acutally boolean?
vmax = 0.2
colmaplabel='Fraction of flux contaminated by neighbors'
#cmap = tab['i_kronRad'][whgood]
#vmax = 20
#colmaplabel='Kron Radius [pixels]'

#ax.scatter(truth_mag,(kron_mag-truth_mag)/truth_mag,color='k', marker='.', alpha=.5)#,vmin=0,vmax=20)
im = ax.scatter(truth_mag[whgood],(kron_mag[whgood]-truth_mag[whgood]),c=cmap, marker='.', alpha=.5,vmin=0,vmax=vmax,label='fromBlend')
ax.axhline(scale,linestyle='--')
ax.set_title('Kron Photometry')

ax.set_xlabel('truth AB Magnitude [i-band]')
ax.set_ylabel('Mag Percent Accuracy (M$_{obs}$ - M$_{true}$) ')
ax.set_xlim([16,26])
ax.set_ylim([-1,1])
ax.legend()

whgood = np.where((tab['i_kronFlux_flag'] == 0) & (tab['i_extendedness'] == 1) & (tab['detect_isIsolated'] == 1) )[0]
#cmap = tab['i_kronRad'][whgood] #tab['i_blendedness'][whgood] # #tab['i_blendedness'][whgood] #
cmap = tab['i_blendedness'][whgood]

#ax2.scatter(truth_mag,(kron_mag-truth_mag)/truth_mag,color='k', marker='.', alpha=.5)#,vmin=0,vmax=20)
ax2.scatter(truth_mag[whgood],(kron_mag[whgood]-truth_mag[whgood]),c=cmap, marker='.', alpha=.5,vmin=0,vmax=vmax,label='isIsolated')
print(len(whgood))
ax2.set_xlabel('truth AB Magnitude [i-band]')
#ax2.set_ylabel('Mag Accuracy (M$_{obs}$ - M$_{true}$) ')
ax2.set_ylim([-1,1])
ax2.set_xlim([16,26])
ax2.legend()
ax2.axhline(scale,linestyle='--')
ax2.set_title('Kron Photometry')

fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
fig.colorbar(im, cax=cbar_ax,label=colmaplabel)


In [ ]:
# the following plots are identical, consider a different comparison that is informative


fig,(ax, ax2) = plt.subplots(1, 2,figsize=(12,5))
#whblend = np.where( tab['detect_fromBlend'][whbright] == False)[0]

# Pick things where there was no flagged error in the Kron meausrement, and things that are extended (i.e. galaxies)

whgood = np.where((tab['i_kronFlux_flag'] == 0) & ( tab['detect_fromBlend'] == 1))[0]

cmap = tab['i_blendedness'][whgood] #only saw things blended with 1 neighbor is this acutally boolean?
vmax = 1
colmaplabel='Fraction of flux contaminated by neighbors'

#cmap = tab['i_kronRad'][whgood]
#vmax = 20
#colmaplabel='Kron Radius [pixels]'

#ax.scatter(truth_mag,(cmodel_mag-truth_mag)/truth_mag,color='k', marker='.', alpha=.5)#,vmin=0,vmax=20)
im = ax.scatter(truth_mag[whgood],(cmodel_mag[whgood]-truth_mag[whgood]),c=cmap, marker='.', alpha=.5,vmin=0,vmax=vmax,label='fromBlend')


ax.set_xlabel('truth AB Magnitude [i-band]')
ax.set_ylabel('Mag Fractional Accuracy (M$_{obs}$ - M$_{true}$) ')
ax.set_ylim([-1,1])
ax.set_xlim([16,26])
ax.axhline(scale,linestyle='--')
ax.set_title('cModel Photometry')
ax.legend()
#cmap = tab['i_kronRad'][whgood] #tab['i_blendedness'][whgood] # #tab['i_blendedness'][whgood] #
whgood = np.where((tab['i_kronFlux_flag'] == 0) & (tab['i_extendedness'] == 1) & (tab['detect_isIsolated'] == 1) )[0]
cmap = tab['i_blendedness'][whgood]

#ax2.scatter(truth_mag,(cmodel_mag-truth_mag),color='k', marker='.', alpha=.5)#,vmin=0,vmax=20)
ax2.scatter(truth_mag[whgood],(cmodel_mag[whgood]-truth_mag[whgood]),c=cmap, marker='.', alpha=.5,vmin=0,vmax=vmax,label='isIsolated')
ax2.set_xlabel('truth AB Magnitude [i-band]')
#ax2.set_ylabel('Mag Accuracy (M$_{obs}$ - M$_{true}$) /  M$_{true}$')
#ax2.set_ylim([-1.5,1.5])
ax2.set_ylim([-1,1])
ax2.set_xlim([16,26])
ax2.axhline(scale,linestyle='--')
print(len(whgood))

fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
fig.colorbar(im, cax=cbar_ax,label=colmaplabel)

ax2.set_title('cModel Photometry')
ax2.legend()

### 2.5 Integrated photometry for high-redshift galaxies

In [ ]:
# perhaps move this to the redshift section 3

zlim = 0.5
whz = np.where(results['redshift'] < zlim)[0]
whighz = np.where(results['redshift'] > zlim)[0]

fig,(ax, ax2) = plt.subplots(1, 2,figsize=(10,5))

ax.axhline(scale,linestyle='--')

ax.plot(truth_mag[whz], (kron_mag[whz]-truth_mag[whz]),'.',alpha=.1,label='Kron',color='blue')
ax.plot(truth_mag[whz], (gaap_mag[whz]-truth_mag[whz]),'.',alpha=.1, label='gaapOptimal',color='green')
ax.plot(truth_mag[whz], (cmodel_mag[whz]-truth_mag[whz]),'.',alpha=.1,label='cModel',color='r')

x = truth_mag[whz]
y = (cmodel_mag[whz]-truth_mag[whz])
bin_means, bin_edges, binnumber = stats.binned_statistic(x,
             y, statistic='median', bins=bins)
binctr = bin_edges[:-1] + (bin_edges[1:]-bin_edges[:-1])/2
ax.plot(binctr, bin_means, color='red', lw=2, label='cModel bin median',zorder=10)

y = (kron_mag[whz]-truth_mag[whz])
bin_means, bin_edges, binnumber = stats.binned_statistic(x,
             y, statistic='median', bins=bins)
binctr = bin_edges[:-1] + (bin_edges[1:]-bin_edges[:-1])/2
ax.plot(binctr, bin_means, color='blue', lw=2, label='kron bin median',zorder=10)

y = (gaap_mag[whz]-truth_mag[whz])
bin_means, bin_edges, binnumber = stats.binned_statistic(x,
             y, statistic='median', bins=bins)
binctr = bin_edges[:-1] + (bin_edges[1:]-bin_edges[:-1])/2
ax.plot(binctr, bin_means, color='green', lw=2, label='GaaP Optimal bin median',zorder=10,linestyle='--')#,color='green')

ax.set_xlabel('True i-band Magnitude')
ax.set_ylabel('Mag Accuracy (M$_{obs}$ - M$_{true}$) ')
#ax.set_ylim([-.2,.2])
ax.set_title('Low redshift z<'+str(zlim))
ax.set_ylim([-2,2])

ax2.axhline(scale,linestyle='--')

ax2.plot(truth_mag[whighz], (kron_mag[whighz]-truth_mag[whighz]),'.',alpha=.1,label='Kron',color='blue')
ax2.plot(truth_mag[whighz], (gaap_mag[whighz]-truth_mag[whighz]),'.',alpha=.1, label='gaapOptimal',color='green')
ax2.plot(truth_mag[whighz], (cmodel_mag[whighz]-truth_mag[whighz]),'.',alpha=.1,label='cModel',color='r')

x = truth_mag[whighz]
y = (cmodel_mag[whighz]-truth_mag[whighz])

bin_means, bin_edges, binnumber = stats.binned_statistic(x,
             y, statistic='median', bins=bins)
binctr = bin_edges[:-1] + (bin_edges[1:]-bin_edges[:-1])/2
ax2.plot(binctr, bin_means, color='red', lw=2, label='cModel bin median',zorder=10)

y = (kron_mag[whighz]-truth_mag[whighz])
bin_means, bin_edges, binnumber = stats.binned_statistic(x,
             y, statistic='median', bins=bins)
binctr = bin_edges[:-1] + (bin_edges[1:]-bin_edges[:-1])/2
ax2.plot(binctr, bin_means, color='blue', lw=2, label='kron bin median',zorder=10)

y = (gaap_mag[whighz]-truth_mag[whighz])
bin_means, bin_edges, binnumber = stats.binned_statistic(x,
             y, statistic='median', bins=bins)
binctr = bin_edges[:-1] + (bin_edges[1:]-bin_edges[:-1])/2
ax2.plot(binctr, bin_means, color='green', lw=2, label='GaaP Optimal bin median',zorder=10,linestyle='--')#,color='green')

ax2.set_xlabel('True i-band Magnitude')
ax2.set_ylim([-2,2])
ax2.set_xlim([19,26])
ax2.set_title('High redshift z>'+str(zlim))
plt.legend()

## 3. Photometry for colors

Here we show observed vs intrinsic colors to demonstrate that GaaP is good for that. Except all 3 measurements are decent and actually cModel has hte least bias (reason, and purpose of GaaP is still TBD).

First, query for the photometry in g and z bands (which will have a large difference in spatial resolution given their relatively large difference wavelengths). u and y band would have an even larger wavelength baseline for comparison, but g and z have better throughput and thus S/N are typically higher. Like earlier sections, also retrieve the truth table input fluxes as a baseline for assessing accuracy in color between the different measurements.

In [ ]:
query = "SELECT mt.id_truth_type, mt.match_objectId, ts.ra, ts.dec, ts.truth_type, ts.flux_g, ts.flux_z, ts.redshift, "\
        "obj.coord_ra, obj.coord_dec, obj.detect_fromBlend, obj.detect_isIsolated, obj.i_blendedness_flag, obj.i_blendedness, obj.i_extendedness, "\
        "obj.g_kronFlux, obj.g_kronFluxErr, obj.g_kronRad, obj.g_kronFlux_flag, obj.g_cModelFlux, obj.g_gaapOptimalFlux, " + \
        "obj.z_kronFlux, obj.z_kronFluxErr, obj.z_kronRad, obj.z_kronFlux_flag, obj.z_cModelFlux, obj.z_gaapOptimalFlux " + \
        "FROM dp02_dc2_catalogs.MatchesTruth AS mt "\
        "JOIN dp02_dc2_catalogs.TruthSummary AS ts ON mt.id_truth_type = ts.id_truth_type "\
        "JOIN dp02_dc2_catalogs.Object AS obj ON mt.match_objectId = obj.objectId "\
        "WHERE (obj.detect_isPrimary = 1) AND (obj.u_kronFlux/obj.u_kronFluxErr > 10)  AND (obj.i_extendedness = 1) AND "\
        "CONTAINS(POINT('ICRS', obj.coord_ra, obj.coord_dec), CIRCLE('ICRS', 62.0, -37.0, 0.20)) = 1 "

In [ ]:
job = service.submit_job(query)
job.run()
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)

results = job.fetch_result()
tab = results.to_table()

In [ ]:
g_cmodel_mag = -2.50 * np.log10(tab['g_cModelFlux']) + 31.4
g_kron_mag = -2.50 * np.log10(tab['g_kronFlux']) + 31.4
g_gaap_mag = -2.50 * np.log10(tab['g_gaapOptimalFlux']) + 31.4
g_truth_mag =  -2.50 * np.log10(tab['flux_g']) + 31.4

z_cmodel_mag = -2.50 * np.log10(tab['z_cModelFlux']) + 31.4
z_kron_mag = -2.50 * np.log10(tab['z_kronFlux']) + 31.4
z_gaap_mag = -2.50 * np.log10(tab['z_gaapOptimalFlux']) + 31.4
z_truth_mag =  -2.50 * np.log10(tab['flux_z']) + 31.4


In [ ]:
# very confused about why these colors line up so perfectly when the mags do not...

fig, (ax, ax2) = plt.subplots(ncols=2, nrows=1, width_ratios=[0.8, 0.2], figsize=(10, 6))
ylims = [-0.2,0.2]

truth_color = g_truth_mag - z_truth_mag
kron_color = g_kron_mag - z_kron_mag
cmodel_color = g_cmodel_mag - z_cmodel_mag
gaap_color =  g_gaap_mag - z_gaap_mag

ax.plot(z_kron_mag, kron_color-truth_color, '.',alpha=.5, label='Kron '+str(np.median(kron_color-truth_color)),color='b')
ax.plot(z_cmodel_mag, cmodel_color-truth_color, '.',alpha=.5, label='cModel '+str(np.median(cmodel_color-truth_color)),color='r')
ax.plot(z_gaap_mag, gaap_color-truth_color, '.',alpha=.5, label='GaaP optimal '+str(np.median(gaap_color-truth_color)),color='g')

ax2.hist((kron_color-truth_color),edgecolor='blue',orientation="horizontal", bins=np.linspace(ylims[0], ylims[1], 80),align='mid',histtype="step",stacked=True,fill=False)
ax2.hist((gaap_color-truth_color),edgecolor='green',orientation="horizontal", bins=np.linspace(ylims[0], ylims[1], 80),align='mid',histtype="step",stacked=True,fill=False)
ax2.hist((cmodel_color-truth_color),edgecolor='red',orientation="horizontal", bins=np.linspace(ylims[0], ylims[1], 80),align='mid',histtype="step",stacked=True,fill=False)
ax2.axhline(0,linestyle='--')


x = np.arange(-.1,6,.1)
#plt.plot(x,x,linestyle='--')
#plt.plot(x+.1,x+.1,linestyle='--')
#plt.plot(x-.1,x-.1,linestyle='--')
ylims = [-1,1]
ax.axhline(0)
ax.set_ylim(ylims)
ax.set_xlim([16,26])
ax.set_xlabel('Observed g-band Magnitude')
ax.set_ylabel('Observed g-z color - True g-z color')
#ax2.set_ylabel('Observed g-z color - True g-z color')
ax.legend()

### 4. Photometry for light profiles (i.e. how to use aperture photometry)
Science application for aperture fluxes
Not sure how to do this yet

In [ ]:
query = "SELECT mt.id_truth_type, mt.match_objectId, ts.ra, ts.dec, ts.truth_type, ts.flux_i, ts.redshift, "\
        "obj.coord_ra, obj.coord_dec, obj.detect_fromBlend, obj.detect_isIsolated, obj.i_blendedness_flag, obj.i_blendedness, obj.i_extendedness, "\
        "obj.i_ap03Flux, obj.i_ap06Flux, obj.i_ap09Flux, obj.i_ap12Flux, obj.i_ap17Flux, obj.i_ap25Flux, obj.i_ap35Flux, obj.i_ap50Flux, obj.i_kronRad, obj.i_kronFlux_flag " + \
        "FROM dp02_dc2_catalogs.MatchesTruth AS mt "\
        "JOIN dp02_dc2_catalogs.TruthSummary AS ts ON mt.id_truth_type = ts.id_truth_type "\
        "JOIN dp02_dc2_catalogs.Object AS obj ON mt.match_objectId = obj.objectId "\
        "WHERE (obj.detect_isPrimary = 1) AND (obj.i_kronFlux/obj.i_kronFluxErr > 10) AND (obj.i_extendedness = 1) AND "\
        "CONTAINS(POINT('ICRS', obj.coord_ra, obj.coord_dec), CIRCLE('ICRS', 62.0, -37.0, 0.10)) = 1 "

In [ ]:
job = service.submit_job(query)
job.run()
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)

In [ ]:
results = job.fetch_result()
tab = results.to_table()

In [ ]:
# pick an object that has large size
wh = np.where(tab['i_kronRad'] > 10)[0]
print(len(wh), tab['i_kronRad'][wh])

rad = np.array([3,6,9,12,17,25,35,50])
light_profile = np.array([tab['i_ap03Flux'][wh][0],tab['i_ap06Flux'][wh][0],tab['i_ap09Flux'][wh][0],tab['i_ap12Flux'][wh][0],tab['i_ap17Flux'][wh][0],
                          tab['i_ap25Flux'][wh][0],tab['i_ap35Flux'][wh][0],tab['i_ap50Flux'][wh][0]])
plt.plot(rad, light_profile, label='Large Radius R='+str(tab['i_kronRad'][wh][0]))
plt.xlabel('Aperture Radius [pixels]')
plt.ylabel('Flux density [nJy]')

# pick an object that has large size
wh = np.where(tab['i_kronRad'] < 5)[0]
print(len(wh), tab['i_kronRad'][wh])

rad = np.array([3,6,9,12,17,25,35,50])
light_profile = np.array([tab['i_ap03Flux'][wh][0],tab['i_ap06Flux'][wh][0],tab['i_ap09Flux'][wh][0],tab['i_ap12Flux'][wh][0],tab['i_ap17Flux'][wh][0],
                          tab['i_ap25Flux'][wh][0],tab['i_ap35Flux'][wh][0],tab['i_ap50Flux'][wh][0]])
plt.plot(rad, light_profile, label='Small Radius R='+str(tab['i_kronRad'][wh][0]))
plt.legend()


In [ ]:
sys.exit()

In [ ]:
#junk that doens't work yet:

# now do the outlier fraction. histogram up the delta_M / M > 0.1 as function of magnitude for the 3

x = truth_mag
y = (cmodel_mag-truth_mag)#/truth_mag
bins=np.arange(16,30,1)
bin_means, bin_edges, binnumber = stats.binned_statistic(x,
             y, statistic=lambda sigmaz: float(len(np.where(np.abs(y) > 0.01)[0]))/len(y) ,bins=bins)

binctr = bin_edges[:-1] + (bin_edges[1:]-bin_edges[:-1])/2
plt.plot(binctr, bin_means, color='red', lw=2, label='cModel bin median',zorder=10)
#plt.hist(bins[:-1],bins,weights=bin_means,color='r',alpha=.5,edgecolor='black', linewidth=1.2)

y = (kron_mag-truth_mag)#/truth_mag
bin_means, bin_edges, binnumber = stats.binned_statistic(x,
             y, statistic=lambda sigmaz: float(len(np.where(y > 0.01)[0]))/len(y) ,bins=bins)
binctr = bin_edges[:-1] + (bin_edges[1:]-bin_edges[:-1])/2
#plt.plot(binctr, bin_means, color='blue', lw=2, label='kron bin median',zorder=10)
#plt.hist(bins[:-1],bins,weights=bin_means,color='b',alpha=.5,edgecolor='black', linewidth=1.2)

y = (gaap_mag-truth_mag)#/truth_mag
bin_means, bin_edges, binnumber = stats.binned_statistic(x,
             y,  statistic=lambda sigmaz: float(len(np.where(y > 0.01)[0]))/len(y) ,bins=bins)
binctr = bin_edges[:-1] + (bin_edges[1:]-bin_edges[:-1])/2
#plt.plot(binctr, bin_means, color='green', lw=2, label='GaaP Optimal bin median',zorder=10)#,color='green')
#plt.hist(bins[:-1],bins,weights=bin_means,color='g',alpha=.5,edgecolor='black', linewidth=1.2)

#ax.scatter(truth_mag,kron_mag,c=tab['i_kronRad'], marker='+', alpha=.5)

#plt.set_xlabel('True i-band Magnitude')
#plt.set_ylabel('Mag Accuracy (M$_{obs}$ - M$_{true}$) /  M$_{true}$')
#ax.set_ylim([-5,5])
#plt.ylim([0,1000])
#ax.set_yscale('log')
plt.legend()


#bin_means, bin_edges, binnumber = stats.binned_statistic(F150mag,sigmaz, statistic=lambda sigmaz: float(len(np.where(sigmaz > 0.15)[0]))/len(sigmaz) ,bins=bins)

print(bin_means)


In [ ]:
# compare kron mag to cmodel mag

#whbright = np.where(tab['i_kronFlux'] > 0.0001e6)[0]
whbright = np.where((tab['i_kronFlux'] > 0.01e6) & (tab['i_kronFlux_flag'] == 0))[0]

cmodel_mag = -2.50 * np.log10(tab['i_cModelFlux']) + 31.4
kron_mag = -2.50 * np.log10(tab['i_kronFlux']) + 31.4

plt.scatter(cmodel_mag,kron_mag,
            c=tab['i_kronRad'], marker='.', alpha=.5)

#plt.scatter(cmodel_mag[whbright],kron_mag[whbright],
#            c=tab['i_kronRad'][whbright], marker='.', alpha=.5)
plt.axhline(26.4,linestyle='--',label='10-yr 5sig depth')

plt.colorbar(label='Kron Radius [pixels]')
plt.xlabel('cModel AB Magnitude [i-band]')
plt.ylabel('kron AB Magnitude [i-band]')
plt.legend()
plt.show()

In [ ]:

cmodel_mag = -2.50 * np.log10(tab['i_gaapOptimalFlux']) + 31.4
kron_mag = -2.50 * np.log10(tab['i_kronFlux']) + 31.4

plt.scatter(cmodel_mag,kron_mag,
            c=tab['i_kronRad'], marker='.', alpha=.5, vmin=1, vmax=50)

#plt.scatter(cmodel_mag[whbright],kron_mag[whbright],
#            c=tab['i_kronRad'][whbright], marker='.', alpha=.5)
plt.axhline(26.4,linestyle='--',label='10-yr 5sig depth')

plt.colorbar(label='Kron Radius [pixels]')
plt.xlabel('GaaP optimal AB Magnitude [i-band]')
plt.ylabel('kron AB Magnitude [i-band]')
plt.legend()
plt.show()

In [ ]:

#cmodel_mag = -2.50 * np.log10(tab['i_gaap1p5Flux']) + 31.4
kron_mag = -2.50 * np.log10(tab['i_kronFlux']) + 31.4

plt.scatter(cmodel_mag,kron_mag,
            c=tab['i_kronRad'], marker='.', alpha=.5, vmin=1, vmax=50)

#plt.scatter(cmodel_mag[whbright],kron_mag[whbright],
#            c=tab['i_kronRad'][whbright], marker='.', alpha=.5)
plt.axhline(26.4,linestyle='--',label='10-yr 5sig depth')

plt.colorbar(label='Kron Radius [pixels]')
plt.xlabel('GaaP AB Magnitude 1.5 aperture [i-band]')
plt.ylabel('kron AB Magnitude [i-band]')
plt.legend()
plt.show()

In [ ]:

whbright = np.where((tab['i_kronFlux'] > 0.001e6) & (tab['i_kronFlux_flag'] == 0))[0]

mag = -2.50 * np.log10(tab['i_kronFlux']) + 31.4
plt.scatter(mag[whbright],tab['i_kronFlux'][whbright]/tab['i_cModelFlux'][whbright],
            c=tab['i_kronRad'][whbright], marker='o', alpha=.5, vmin=1, vmax=50)
plt.colorbar(label='Kron Radius [pixels]')
plt.axhline(1,linestyle='--')
plt.xlabel('Kron AB Magnitude [i-band]')
plt.yscale('log')
plt.ylabel('Kron flux / cModel Flux')


whbad = np.where( (np.abs(tab['i_kronFlux'][whbright]/tab['i_cModelFlux'][whbright]) > 2) & (mag[whbright] < 22))[0]
whblend = np.where( tab['detect_fromBlend'][whbright] == True)[0]

print(tab['detect_fromBlend'][whbright][whbad])
plt.plot(mag[whbright][whblend],tab['i_kronFlux'][whbright][whblend]/tab['i_cModelFlux'][whbright][whblend],'k+')

plt.show()

In [ ]:
plt.scatter(tab['i_kronRad'][whbright],tab['i_kronFlux'][whbright]/tab['i_gaapOptimalFlux'][whbright],
            c=mag[whbright], marker='o', alpha=.5)
plt.xlabel('Kron Radius [pix]')
plt.ylabel('Kron flux / GaaP Optimal Flux')
plt.colorbar(label='Kron Mag [ABmag]')
plt.xlim([0,50])
plt.ylim([-1,30])
plt.show()

In [ ]:
plt.scatter(tab['i_kronRad'][whbright],tab['i_kronFlux'][whbright]/tab['i_cModelFlux'][whbright],
            c=mag[whbright], marker='o', alpha=.5)
plt.xlabel('Kron Radius [pix]')
plt.ylabel('Kron flux / cModel Flux')
plt.colorbar(label='Kron Mag [ABmag]')
plt.xlim([0,50])
plt.ylim([-1,30])
plt.show()

# 4. AGN and host photometry

easier for someone to correct something i've done wrong than to tell me the right way to do it (so, try to write it up and take af irst stab at it and then ask if thats what they were going for). 

for DP1: here is literal interpretation of measurement (these should be short)

then one for low-z galaxies and nuances of measuring photometry

then one for high-z galaxies 

AGN: point source right in the center thats a better science case than the SN contamination one. there will be more of these and stacking doesn't take it out.

can we think of a science case we would want to use galaxy photometry from a calexp? this is probably a different science case. lets exclude calexp photometry form this notebook. 

when you're ready for input from DM make a list of what i want help with (so Yusra can find someone for help and then help them respond to specific questions